# Analyzing Cisplatin-Binding Transcripts via GLM-SAE Knowledge Graphs

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph

import os, json

with open("data/cisplatin_neg45k_kg.json") as f:
    data = json.load(f)

G: nx.MultiDiGraph = json_graph.node_link_graph(data, edges="edges")
print(G)

MultiDiGraph with 7141 nodes and 4197345 edges


In [2]:
# graph is large, so lets trim down a version we can visualize
OG = G.copy()

# filter features that only respond to one token
to_remove = [n for n in G.nodes if G.in_degree(n) == 1]
G.remove_nodes_from(to_remove)
print(G)

# the '<cls>' token is the 'start' token and does not encode genetic info
if G.has_node('<cls>'):
    G.remove_node('<cls>')
    print(G)

# find average edge bundle size
edge_sizes = {(u,v): len(G.get_edge_data(u, v)) for u, v, k in G.edges}
avg_edge_size = sum(edge_sizes.values()) / len(edge_sizes)
print(f"Average edge bundle size: {avg_edge_size}")
print(f"Median edge bundle size: {np.median(list(edge_sizes.values()))}")
print(f"Number of edge bundles: {len(edge_sizes)}")
print(f"Max edge bundle size: {max(edge_sizes.values())}")
print(f"Min edge bundle size: {min(edge_sizes.values())}")
print(f"Standard deviation of edge bundle sizes: {pd.Series(edge_sizes.values()).std()}")

# filter edges that have less than 5 samples in their bundle, then show new descriptive stats
print("---------- Filtering edges with less than 5 samples in their bundle ----------")
to_remove = [key for key, size in edge_sizes.items() if size < 5]
for ebundle in to_remove:
    u, v = ebundle
    keys = list(G.get_edge_data(u, v).keys())
    for k in keys:
        G.remove_edge(u, v, k)
# G.remove_edges_from(to_remove)
print(G)

edge_sizes = {(u,v): len(G.get_edge_data(u, v)) for u, v, k in G.edges}
avg_edge_size = sum(edge_sizes.values()) / len(edge_sizes)
print(f"Average edge bundle size: {avg_edge_size}")
print(f"Median edge bundle size: {np.median(list(edge_sizes.values()))}")
print(f"Number of edge bundles: {len(edge_sizes)}")
print(f"Max edge bundle size: {max(edge_sizes.values())}")
print(f"Min edge bundle size: {min(edge_sizes.values())}")
print(f"Standard deviation of edge bundle sizes: {pd.Series(edge_sizes.values()).std()}")

# re-filter features that only respond to one token or are disconnected from the graph
to_remove = [n for n in G.nodes if G.in_degree(n) <= 1 and type(n) == int]
G.remove_nodes_from(to_remove)
print(G)

MultiDiGraph with 6824 nodes and 4197028 edges
MultiDiGraph with 6823 nodes and 4156725 edges
Average edge bundle size: 6.9515217598531
Median edge bundle size: 1.0
Number of edge bundles: 597959
Max edge bundle size: 6057
Min edge bundle size: 1
Standard deviation of edge bundle sizes: 46.394946770635855
---------- Filtering edges with less than 5 samples in their bundle ----------
MultiDiGraph with 6823 nodes and 3387035 edges
Average edge bundle size: 34.867562281243565
Median edge bundle size: 9.0
Number of edge bundles: 97140
Max edge bundle size: 6057
Min edge bundle size: 5
Standard deviation of edge bundle sizes: 110.97611840464725
MultiDiGraph with 5568 nodes and 3387035 edges


In [3]:
# use annotations to create two graphs of coding and non-coding genomic regions
coding_regions = [e for e in G.edges(data=True, keys=True) if len(e[3]["annotations"]) != 0]
non_coding_regions = [e for e in G.edges(data=True, keys=True) if len(e[3]["annotations"]) == 0]

G_coding = nx.MultiDiGraph()
for edge in coding_regions:
    G_coding.add_edge(edge[0], edge[1], edge[2], **edge[3])
G_non_coding = nx.MultiDiGraph()
for edge in non_coding_regions:
    G_non_coding.add_edge(edge[0], edge[1], edge[2], **edge[3])

print(f"Coding graph: {G_coding}")
print(f"Non-coding graph: {G_non_coding}")

Coding graph: MultiDiGraph with 5415 nodes and 812369 edges
Non-coding graph: MultiDiGraph with 5567 nodes and 2574666 edges


In [4]:
def node_color(x):
    return "lightblue" if type(x) == str else "red"

In [5]:
print("--- CODING SEQUENCES ---")

pagerank = nx.pagerank(G_coding)
sorted_pr = sorted(pagerank.items(), key=lambda item: item[1], reverse=True)
print("Top 10 nodes by PageRank:")
for node, pr in sorted_pr[:10]:
    print(f"Node: {node}, PageRank: {pr}")

degree_centrality = nx.out_degree_centrality(G_coding)
sorted_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)
print("Top 10 nodes by Out-Degree Centrality:")
for node, dc in sorted_degree[:10]:
    print(f"Node: {node}, Out-Degree Centrality: {dc}")

#nx.draw(G_coding, pos=nx.forceatlas2_layout(G_coding), with_labels=False, node_size=10, font_size=4, node_color=[node_color(n) for n in G_coding.nodes()])

--- CODING SEQUENCES ---
Top 10 nodes by PageRank:
Node: 8161, PageRank: 0.19687593503427994
Node: 7030, PageRank: 0.0372370455963754
Node: 7949, PageRank: 0.014281489531611225
Node: 1422, PageRank: 0.007482991577658241
Node: 5984, PageRank: 0.006915740386288211
Node: 2453, PageRank: 0.0048780476203867185
Node: 9452, PageRank: 0.004333574960662349
Node: 9021, PageRank: 0.003083666406978392
Node: 835, PageRank: 0.002501853543098789
Node: 5741, PageRank: 0.0024684622382828867
Top 10 nodes by Out-Degree Centrality:
Node: TTTTTT, Out-Degree Centrality: 0.25434059844846696
Node: AAAAAA, Out-Degree Centrality: 0.2133357960842261
Node: ATTTTT, Out-Degree Centrality: 0.18729220539342445
Node: TTTTTA, Out-Degree Centrality: 0.17713335796084226
Node: G, Out-Degree Centrality: 0.16715921684521612
Node: TATTTT, Out-Degree Centrality: 0.16642039157739194
Node: AAAAAT, Out-Degree Centrality: 0.16512744735869966
Node: TTTTTG, Out-Degree Centrality: 0.16217214628740304
Node: TTTTAA, Out-Degree Central

In [6]:
print("--- NON-CODING SEQUENCES ---")

pagerank = nx.pagerank(G_non_coding)
sorted_pr = sorted(pagerank.items(), key=lambda item: item[1], reverse=True)
print("Top 10 nodes by PageRank:")
for node, pr in sorted_pr[:10]:
    print(f"Node: {node}, PageRank: {pr}")

degree_centrality = nx.out_degree_centrality(G_non_coding)
sorted_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)
print("Top 10 nodes by Out-Degree Centrality:")
for node, dc in sorted_degree[:10]:
    print(f"Node: {node}, Out-Degree Centrality: {dc}")


#nx.draw(G_non_coding, pos=nx.forceatlas2_layout(G_non_coding), with_labels=False, node_size=10, font_size=4, node_color=[node_color(n) for n in G_non_coding.nodes()])

--- NON-CODING SEQUENCES ---
Top 10 nodes by PageRank:
Node: 8161, PageRank: 0.16845245564335232
Node: 7030, PageRank: 0.029691228481648322
Node: 3378, PageRank: 0.012824458852998533
Node: 4793, PageRank: 0.0095631601344086
Node: 7949, PageRank: 0.006341539512496292
Node: 1422, PageRank: 0.006265641335004452
Node: 5984, PageRank: 0.00532525965976957
Node: 1545, PageRank: 0.004422141159927955
Node: 835, PageRank: 0.0043350835181561135
Node: 7612, PageRank: 0.004331091308329926
Top 10 nodes by Out-Degree Centrality:
Node: G, Out-Degree Centrality: 1.7919511318720804
Node: C, Out-Degree Centrality: 1.7642831476823573
Node: A, Out-Degree Centrality: 1.6658282429033417
Node: T, Out-Degree Centrality: 1.5449155587495509
Node: AAAAAA, Out-Degree Centrality: 0.8841178584261589
Node: TTTTTT, Out-Degree Centrality: 0.8514193316564859
Node: AAAAAT, Out-Degree Centrality: 0.5979159180740209
Node: ATTTTT, Out-Degree Centrality: 0.5725835429392742
Node: TAAAAA, Out-Degree Centrality: 0.5415019762845

In [7]:
# COMPARE TO ORIGINAL GRAPH
# use page rank to measure centrality
pagerank = nx.pagerank(OG)
sorted_pagerank = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)

print("Nodes ranked by PageRank:")
for node, score in sorted_pagerank[:10]:  # top 10 nodes
    print(f"{node}: {score}")

Nodes ranked by PageRank:
8161: 0.10874592750416887
7030: 0.021950368456290115
3378: 0.007113011140264806
4793: 0.005248423700916205
7949: 0.004543801336913997
5984: 0.004379874340400829
1422: 0.00405684001443536
2453: 0.0032858460917511473
7612: 0.002891826775626524
835: 0.0028105569144162477


In [8]:
# COMPARE TO ORIGINAL GRAPH
# compare with degree centrality for important tokens
degree_centrality = nx.out_degree_centrality(OG)
sorted_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)

print("Nodes ranked by Degree Centrality:")
for node, score in sorted_degree[:10]:  # top 10 nodes
    print(f"{node}: {score}")

Nodes ranked by Degree Centrality:
<cls>: 5.644677871148459
G: 1.6274509803921569
C: 1.5823529411764705
A: 1.4928571428571427
T: 1.3866946778711484
TTTTTT: 0.8684873949579831
AAAAAA: 0.8638655462184873
ATTTTT: 0.6273109243697479
AAAAAT: 0.6194677871148458
TAAAAA: 0.5546218487394957


In [9]:
# export to cytoscape for visualization
# Didn't work for the large graph, so just keeping for documentation
for node in OG.nodes():
    OG.nodes[node]['id'] = node

cyto_data = nx.readwrite.json_graph.cytoscape_data(OG)
with open("data/cisplatin_neg45k_kg_cyto.json", "w") as f:
    json.dump(cyto_data, f)

# Using Knowledge Graph Patterns to Create Gene Sets

Note the strong centrality in multiple segments of the graph for features 3378, 407, and 4793, as well as typical base pairing hairpin motifs with high GC content in the outbound central nodes. 

By gathering data from the edges associated with each major feature, we can create gene sets based on GLM feature output. 

In [10]:
# Note that our non-coding graph lacks annotations by definition, so we will only create gene sets for coding regions
# central feat 0
best_feature = sorted_pagerank[0][0]
best_feature2 = sorted_pagerank[1][0]
best_feature3 = sorted_pagerank[2][0]

edges = G_coding.in_edges(best_feature, data="annotations")

genes = []
motifs = []
for u, v, annotations in edges:
    for key, val in annotations.items():
        genes.append(val["gene_id"])
        motifs.append(u)

os.makedirs("data/neg_gene_sets", exist_ok=True)

with open(f"data/neg_gene_sets/feature_{best_feature}_genes.txt", "w") as f:
    f.write("\n".join(set(genes)))

with open(f"data/neg_gene_sets/feature_{best_feature}_motifs.txt", "w") as f:
    f.write("\n".join(set(motifs)))

# 407
edges = G_coding.in_edges(best_feature2, data="annotations")

genes = []
motifs = []
for u, v, annotations in edges:
    for key, val in annotations.items():
        genes.append(val["gene_id"])
        motifs.append(u)

with open(f"data/neg_gene_sets/feature_{best_feature2}_genes.txt", "w") as f:
    f.write("\n".join(set(genes)))

with open(f"data/neg_gene_sets/feature_{best_feature2}_motifs.txt", "w") as f:
    f.write("\n".join(set(motifs)))


# 4793
edges = G_coding.in_edges(best_feature3, data="annotations")

genes = []
motifs = []
for u, v, annotations in edges:
    for key, val in annotations.items():
        genes.append(val["gene_id"])
        motifs.append(u)

with open(f"data/neg_gene_sets/feature_{best_feature3}_genes.txt", "w") as f:
    f.write("\n".join(set(genes)))

with open(f"data/neg_gene_sets/feature_{best_feature3}_motifs.txt", "w") as f:
    f.write("\n".join(set(motifs)))


# overall gene sets for comparison
edges = G_coding.edges(data="annotations")
genes = []
motifs = []
for u, v, annotations in edges:
    for key, val in annotations.items():
        genes.append(val["gene_id"])
        motifs.append(u)
with open("data/neg_gene_sets/all_coding_genes.txt", "w") as f:
    f.write("\n".join(set(genes)))
with open("data/neg_gene_sets/all_coding_motifs.txt", "w") as f:
    f.write("\n".join(set(motifs)))

In [11]:
# free up memory
del G
del G_coding
del G_non_coding
del cyto_data
del edges
del genes
del motifs
del sorted_pagerank
del sorted_pr

# Community Detection

In [12]:
# Louvain
communities = nx.community.louvain_communities(OG.to_undirected(), resolution=1.5)
print(f"Found {len(communities)} communities")

Found 43 communities


In [13]:
subgraph_list = []
for i, c in enumerate(communities):
    graph = OG.subgraph(c)
    central_features = sorted(nx.in_degree_centrality(graph).items(), key=lambda x: x[1], reverse=True)[:5]
    central_motifs = sorted(nx.out_degree_centrality(graph).items(), key=lambda x: x[1], reverse=True)[:5]

    print(f"Community {i}:")
    print(" Central Features (In-Degree Centrality):")
    for node, cent in central_features:
        print(f"  Node: {node}, In-Degree Centrality: {cent}")
    print(" Central Motifs (Out-Degree Centrality):")
    for node, cent in central_motifs:
        print(f"  Node: {node}, Out-Degree Centrality: {cent}")
    graph = graph.to_undirected()

    # Export each community to a separate JSON file
    if not os.path.exists(f"data/negKG_louvain_communities/{i}"):
        os.makedirs(f"data/negKG_louvain_communities/{i}", exist_ok=True)
    with open(f"data/negKG_louvain_communities/{i}/graph.json", "w") as f:
        json.dump(json_graph.node_link_data(graph, edges="edges"), f)

    subgraph_list.append({"g":graph, "central_feat": central_features[0], "central_motif": central_motifs[0]})
    print("")
    # Visualize the community
#    nx.draw(graph, pos=nx.kamada_kawai_layout(graph), with_labels=False, node_size=10, font_size=4, node_color=[node_color(n) for n in graph.nodes()])
#    plt.title(f"Community {i}")
#    plt.show()

Community 0:
 Central Features (In-Degree Centrality):
  Node: 5746, In-Degree Centrality: 35124.0
  Node: <cls>, In-Degree Centrality: 0.0
 Central Motifs (Out-Degree Centrality):
  Node: <cls>, Out-Degree Centrality: 35124.0
  Node: 5746, Out-Degree Centrality: 0.0

Community 1:
 Central Features (In-Degree Centrality):
  Node: 7030, In-Degree Centrality: 323.2313432835821
  Node: 2471, In-Degree Centrality: 6.384328358208955
  Node: 720, In-Degree Centrality: 5.527985074626866
  Node: 1003, In-Degree Centrality: 4.227611940298507
  Node: 954, In-Degree Centrality: 4.210820895522388
 Central Motifs (Out-Degree Centrality):
  Node: AGAAAA, Out-Degree Centrality: 5.975746268656716
  Node: TTTTTA, Out-Degree Centrality: 5.656716417910448
  Node: AAGAAA, Out-Degree Centrality: 5.5783582089552235
  Node: TTTTAA, Out-Degree Centrality: 5.317164179104478
  Node: ATTTTT, Out-Degree Centrality: 5.255597014925373

Community 2:
 Central Features (In-Degree Centrality):
  Node: 1845, In-Degree C

In [14]:
# generate gene sets for central features and motifs in each community
for i, sub in enumerate(subgraph_list):
    graph = sub["g"]
    central_feat = sub["central_feat"][0]
    central_motif = sub["central_motif"][0]

    # central feature
    if type(central_feat) == int:
        edges = graph.edges(central_feat, data="annotations")
        genes = []
        motifs = []
        for u, v, annotations in edges:
            for key, val in annotations.items():
                motif = u if u != central_feat else v
                genes.append(val["gene_id"])
                motifs.append(motif)
        with open(f"data/negKG_louvain_communities/{i}/feature_{central_feat}_genes.txt", "w") as f:
            f.writelines('\n'.join(set(genes)))
        with open(f"data/negKG_louvain_communities/{i}/feature_{central_feat}_motifs.txt", "w") as f:
            f.writelines('\n'.join(set(motifs)))

    # central motif
    if type(central_motif) == str:
        edges = graph.edges(central_motif, data="annotations")
        genes = []
        features = []
        for u, v, annotations in edges:
            for key, val in annotations.items():
                feature = u if u != central_motif else v
                genes.append(val["gene_id"])
                features.append(str(feature))
        with open(f"data/negKG_louvain_communities/{i}/motif_{central_motif}_genes.txt", "w") as f:
            f.writelines('\n'.join(set(genes)))
        with open(f"data/negKG_louvain_communities/{i}/motif_{central_motif}_features.txt", "w") as f:
            f.writelines('\n'.join(set(features)))

# Gene Set Enrichment Analysis

In [15]:
# run R enrichment analysis on these gene sets using our custom script

neg_gene_sets = [f for f in os.listdir("data/neg_gene_sets") if f.endswith("_genes.txt")]
for gs in neg_gene_sets:
    os.system(f"Rscript enrich.R --gene_list data/neg_gene_sets/{gs} --outdir data/neg_gene_sets/enrich_{gs.split('_')[1]}")

for c in os.listdir("data/negKG_louvain_communities"):
    neg_gene_sets = [f for f in os.listdir(f"data/negKG_louvain_communities/{c}") if f.endswith("_genes.txt")]
    for gs in neg_gene_sets:
        os.system(f"Rscript enrich.R --gene_list data/negKG_louvain_communities/{c}/{gs} --outdir data/negKG_louvain_communities/{c}/enrich_{gs.split('_')[1]}")


clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

G Yu. Thirteen years of clusterProfiler. The Innovation. 2024,
5(6):100722

Attaching package: ‘clusterProfiler’

The following object is masked from ‘package:stats’:

    filter

enrichplot v1.28.4 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

S Xu, E Hu, Y Cai, Z Xie, X Luo, L Zhan, W Tang, Q Wang, B Liu, R Wang,
W Xie, T Wu, L Xie, G Yu. Using clusterProfiler to characterize
multiomics data. Nature Protocols. 2024, 19(11):3292-3320
Welcome to enrichR
Checking connections ... 
Enrichr ... Connection is Live!
FlyEnrichr ... Connection is Live!
WormEnrichr ... Connection is Live!
YeastEnrichr ... Connection is Live!
FishEnrichr ... Connection is Live!
OxEnrichr ... Connection is Live!

Attaching package: ‘generics’

The following objects are masked from ‘package:base’:

    as.difftime, as.factor, as.ordered, intersect, is.element, setdiff,
    

[1] "Reading gene list from: data/neg_gene_sets/feature_8161_genes.txt"
[1] "Performing GO enrichment analysis for ontology: BP"
[1] "Generating plots..."
[1] "Saving results to directory: data/neg_gene_sets/enrich_8161"


Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 

clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

S Xu, E Hu, Y Cai, Z Xie, X Luo, L Zhan, W Tang, Q Wang, B Liu, R Wang,
W Xie, T Wu, L Xie, G Yu. Using clusterProfiler to characterize
multiomics data. Nature Protocols.

[1] "Reading gene list from: data/neg_gene_sets/feature_3378_genes.txt"
[1] "Performing GO enrichment analysis for ontology: BP"
[1] "Generating plots..."
[1] "Saving results to directory: data/neg_gene_sets/enrich_3378"


Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 

clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

Guangchuang Yu, Li-Gen Wang, Yanyan Han and Qing-Yu He.
clusterProfiler: an R package for comparing biological themes among
gene clusters. OMICS: A Journal of Integrative

...

Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 

clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan,
X Fu, S Liu, X Bo, and G Yu. clusterProfiler 4.0: A universal
enrichment tool for interpreting omi

[1] "Reading gene list from: data/negKG_louvain_communities/24/feature_526_genes.txt"
[1] "Performing GO enrichment analysis for ontology: BP"
[1] "Generating plots..."
[1] "Saving results to directory: data/negKG_louvain_communities/24/enrich_526"


Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 

clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

Guangchuang Yu, Li-Gen Wang, Yanyan Han and Qing-Yu He.
clusterProfiler: an R package for comparing biological themes among
gene clusters. OMICS: A Journal of Integrative

[1] "Reading gene list from: data/negKG_louvain_communities/24/motif_GCCTCC_genes.txt"
[1] "Performing GO enrichment analysis for ontology: BP"
[1] "Generating plots..."
[1] "Saving results to directory: data/negKG_louvain_communities/24/enrich_GCCTCC"


Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 


In [16]:
# molecular function enrichment as well

neg_gene_sets = [f for f in os.listdir("data/neg_gene_sets") if f.endswith("_genes.txt")]
for gs in neg_gene_sets:
    os.system(f"Rscript enrich.R --gene_list data/neg_gene_sets/{gs} --outdir data/neg_gene_sets/enrich_{gs.split('_')[1]} --ont MF")

for c in os.listdir("data/negKG_louvain_communities"):
    neg_gene_sets = [f for f in os.listdir(f"data/negKG_louvain_communities/{c}") if f.endswith("_genes.txt")]
    for gs in neg_gene_sets:
        os.system(f"Rscript enrich.R --gene_list data/negKG_louvain_communities/{c}/{gs} --outdir data/negKG_louvain_communities/{c}/enrich_{gs.split('_')[1]} --ont MF")


clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

G Yu. Thirteen years of clusterProfiler. The Innovation. 2024,
5(6):100722

Attaching package: ‘clusterProfiler’

The following object is masked from ‘package:stats’:

    filter

enrichplot v1.28.4 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

Guangchuang Yu, Li-Gen Wang, Guang-Rong Yan, Qing-Yu He. DOSE: an
R/Bioconductor package for Disease Ontology Semantic and Enrichment
analysis. Bioinformatics. 2015, 31(4):608-609
Welcome to enrichR
Checking connections ... 
Enrichr ... Connection is Live!
FlyEnrichr ... Connection is Live!
WormEnrichr ... Connection is Live!
YeastEnrichr ... Connection is Live!
FishEnrichr ... Connection is Live!
OxEnrichr ... Connection is Live!

Attaching package: ‘generics’

The following objects are masked from ‘package:base’:

    as.difftime, as.factor, as.ordered, intersect, is.element, setdiff,
    setequal, unio

[1] "Reading gene list from: data/neg_gene_sets/feature_8161_genes.txt"
[1] "Performing GO enrichment analysis for ontology: MF"
[1] "Generating plots..."
[1] "Saving results to directory: data/neg_gene_sets/enrich_8161"


Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 

clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan,
X Fu, S Liu, X Bo, and G Yu. clusterProfiler 4.0: A universal
enrichment tool for interpreting omi

[1] "Reading gene list from: data/neg_gene_sets/feature_3378_genes.txt"
[1] "Performing GO enrichment analysis for ontology: MF"
[1] "Generating plots..."
[1] "Saving results to directory: data/neg_gene_sets/enrich_3378"


Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 

clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

Guangchuang Yu, Li-Gen Wang, Yanyan Han and Qing-Yu He.
clusterProfiler: an R package for comparing biological themes among
gene clusters. OMICS: A Journal of Integrative

...


clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan,
X Fu, S Liu, X Bo, and G Yu. clusterProfiler 4.0: A universal
enrichment tool for interpreting omics data. The Innovation. 2021,
2(3):100141

Attaching package: ‘clusterProfiler’

The following object is masked from ‘package:stats’:

    filter

enrichplot v1.28.4 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

Guangchuang Yu, Li-Gen Wang, Yanyan Han and Qing-Yu He.
clusterProfiler: an R package for comparing biological themes among
gene clusters. OMICS: A Journal of Integrative Biology. 2012,
16(5):284-287
Welcome to enrichR
Checking connections ... 
Enrichr ... Connection is Live!
FlyEnrichr ... Connection is Live!
WormEnrichr ... Connection is Live!
YeastEnrichr ... Connection is Live!
FishEnrichr ... Connection is Live!
OxEnrichr ... Connection is Live!

Attaching package:

[1] "Reading gene list from: data/negKG_louvain_communities/24/feature_526_genes.txt"
[1] "Performing GO enrichment analysis for ontology: MF"
[1] "Generating plots..."
[1] "Saving results to directory: data/negKG_louvain_communities/24/enrich_526"


Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 

clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

G Yu. Thirteen years of clusterProfiler. The Innovation. 2024,
5(6):100722

Attaching package: ‘clusterProfiler’

The following object is masked from ‘package:stats’:

  

[1] "Reading gene list from: data/negKG_louvain_communities/24/motif_GCCTCC_genes.txt"
[1] "Performing GO enrichment analysis for ontology: MF"
[1] "Generating plots..."
[1] "Saving results to directory: data/negKG_louvain_communities/24/enrich_GCCTCC"


Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 


In [17]:
# cellular component enrichment as well

neg_gene_sets = [f for f in os.listdir("data/neg_gene_sets") if f.endswith("_genes.txt")]
for gs in neg_gene_sets:
    os.system(f"Rscript enrich.R --gene_list data/neg_gene_sets/{gs} --outdir data/neg_gene_sets/enrich_{gs.split('_')[1]} --ont CC")

for c in os.listdir("data/negKG_louvain_communities"):
    neg_gene_sets = [f for f in os.listdir(f"data/negKG_louvain_communities/{c}") if f.endswith("_genes.txt")]
    for gs in neg_gene_sets:
        os.system(f"Rscript enrich.R --gene_list data/negKG_louvain_communities/{c}/{gs} --outdir data/negKG_louvain_communities/{c}/enrich_{gs.split('_')[1]} --ont CC")


clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan,
X Fu, S Liu, X Bo, and G Yu. clusterProfiler 4.0: A universal
enrichment tool for interpreting omics data. The Innovation. 2021,
2(3):100141

Attaching package: ‘clusterProfiler’

The following object is masked from ‘package:stats’:

    filter

enrichplot v1.28.4 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

Guangchuang Yu, Qing-Yu He. ReactomePA: an R/Bioconductor package for
reactome pathway analysis and visualization. Molecular BioSystems.
2016, 12(2):477-479
Welcome to enrichR
Checking connections ... 
Enrichr ... Connection is Live!
FlyEnrichr ... Connection is Live!
WormEnrichr ... Connection is Live!
YeastEnrichr ... Connection is Live!
FishEnrichr ... Connection is Live!
OxEnrichr ... Connection is Live!

Attaching package: ‘generics’

The following objects are mask

[1] "Reading gene list from: data/neg_gene_sets/feature_8161_genes.txt"
[1] "Performing GO enrichment analysis for ontology: CC"
[1] "Generating plots..."
[1] "Saving results to directory: data/neg_gene_sets/enrich_8161"


Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 

clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

Guangchuang Yu, Li-Gen Wang, Yanyan Han and Qing-Yu He.
clusterProfiler: an R package for comparing biological themes among
gene clusters. OMICS: A Journal of Integrative

[1] "Reading gene list from: data/neg_gene_sets/feature_3378_genes.txt"
[1] "Performing GO enrichment analysis for ontology: CC"
[1] "Generating plots..."
[1] "Saving results to directory: data/neg_gene_sets/enrich_3378"


Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 

clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

G Yu. Thirteen years of clusterProfiler. The Innovation. 2024,
5(6):100722

Attaching package: ‘clusterProfiler’

The following object is masked from ‘package:stats’:

  

...


clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

Guangchuang Yu, Li-Gen Wang, Yanyan Han and Qing-Yu He.
clusterProfiler: an R package for comparing biological themes among
gene clusters. OMICS: A Journal of Integrative Biology. 2012,
16(5):284-287

Attaching package: ‘clusterProfiler’

The following object is masked from ‘package:stats’:

    filter

enrichplot v1.28.4 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

Guangchuang Yu. Using meshes for MeSH term enrichment and semantic
analyses. Bioinformatics. 2018, 34(21):3766-3767,
doi:10.1093/bioinformatics/bty410
Welcome to enrichR
Checking connections ... 
Enrichr ... Connection is Live!
FlyEnrichr ... Connection is Live!
WormEnrichr ... Connection is Live!
YeastEnrichr ... Connection is Live!
FishEnrichr ... Connection is Live!
OxEnrichr ... Connection is Live!

Attaching package: ‘generics’

The following objects are masked from ‘package:ba

[1] "Reading gene list from: data/negKG_louvain_communities/24/feature_526_genes.txt"
[1] "Performing GO enrichment analysis for ontology: CC"
[1] "Generating plots..."
[1] "Saving results to directory: data/negKG_louvain_communities/24/enrich_526"


Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 

clusterProfiler v4.16.0 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan,
X Fu, S Liu, X Bo, and G Yu. clusterProfiler 4.0: A universal
enrichment tool for interpreting omi

[1] "Reading gene list from: data/negKG_louvain_communities/24/motif_GCCTCC_genes.txt"
[1] "Performing GO enrichment analysis for ontology: CC"
[1] "Generating plots..."
[1] "Saving results to directory: data/negKG_louvain_communities/24/enrich_GCCTCC"


Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
ℹ The deprecated feature was likely used in the ggtangle package.
  Please report the issue to the authors. 
2: In fortify(object, showCategory = showCategory, by = x, ...) :
  Arguments in `...` must be used.
✖ Problematic argument:
• by = x
ℹ Did you misspell an argument name?
3: `aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.
ℹ The deprecated feature was likely used in the enrichplot package.
  Please report the issue at
  <https://github.com/GuangchuangYu/enrichplot/issues>. 
